In [6]:
import pandas as pd
import numpy as np
import os
os.getcwd()

'/Users/praveen/MSDS/MachineLearning/KaggleChallenges/Challenge2/Data'

In [7]:
train = pd.read_csv("./train_data_new.csv")
test = pd.read_csv("./test_data_new.csv")

/var/folders/h5/yfvk0l4921v0kg94n8qt42wc0000gn/T/ipykernel_65768/2922911961.py:1: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("./train_data_new.csv")


In [8]:
train = pd.read_csv("./train_data_new.csv")
test = pd.read_csv("./test_data_new.csv")
cat_vars = ['general_sector', 'city', 'zip_code', 'specific_sector', 'state',
       'contributor_type']
train_num_vars = ['winner_ratio', 'candidacy_count',
       'candidacy_democratic_count', 'candidacy_republican_count',
       'contribution_count', 'contribution_democratic_count',
       'contribution_republican_count', 'politician_challenger_count',
       'politician_count', 'politician_democratic_count',
       'politician_incumbency_count', 'politician_open_pos_count',
       'politician_republican_count', 'contribution_democratic_sum_2010_usd',
       'contribution_republican_sum_2010_usd', 'contribution_sum_2010_usd',
       'governor_contributions_sum_2010_usd',
       'house_and_assembly_contributions_sum_2010_usd',
       'senate_contributions_sum_2010_usd',
       'us_house_contributions_sum_2010_usd',
       'us_senate_contributions_sum_2010_usd', 'candidacy_democratic_ratio',
       'candidacy_republican_ratio', 'contribution_democratic_count_ratio',
       'contribution_republican_count_ratio', 'governor_contribution_ratio',
       'house_and_assembly_contribution_ratio', 'politician_challenger_ratio',
       'politician_democratic_ratio', 'politician_incumbency_ratio',
       'politician_open_pos_ratio', 'politician_republican_ratio',
       'senate_contribution_ratio', 'us_house_contribution_ratio',
       'us_senate_contribution_ratio', 'contrib_state_deg',
       'contrib_state_wdeg', 'contrib_state_pr', 'contrib_state_comm',
       'contrib_fed_deg', 'contrib_fed_wdeg', 'contrib_fed_pr',
       'contrib_fed_comm', 'state_all_deg', 'state_all_wdeg', 'state_all_pr',
       'state_all_comm', 'state_win_deg', 'state_win_wdeg', 'state_win_pr',
       'state_win_comm']
test_num_vars = [col for col in train_num_vars if col != "winner_ratio"]

train[train_num_vars] = train[train_num_vars].apply(pd.to_numeric, errors='coerce')
test[test_num_vars] = test[test_num_vars].apply(pd.to_numeric, errors='coerce')
cols_with_negatives = ['contribution_democratic_sum_2010_usd','contribution_republican_sum_2010_usd','contribution_sum_2010_usd','governor_contributions_sum_2010_usd',
'house_and_assembly_contributions_sum_2010_usd','senate_contributions_sum_2010_usd','us_house_contributions_sum_2010_usd',
'us_senate_contributions_sum_2010_usd','governor_contribution_ratio','house_and_assembly_contribution_ratio','senate_contribution_ratio',
'us_house_contribution_ratio','us_senate_contribution_ratio']

### Treat these descrepent columns
for col in cols_with_negatives:
    train[col] = train[col].clip(lower=0)
    test[col] = test[col].clip(lower=0)

import numpy as np

# 1) Define your ratio definitions (old_name, numerator, denominator)
ratio_defs = [
    # count‑based
    ("candidacy_democratic_ratio",   "candidacy_democratic_count",            "candidacy_count"),
    ("candidacy_republican_ratio",   "candidacy_republican_count",            "candidacy_count"),
    ("contribution_democratic_count_ratio","contribution_democratic_count","contribution_count"),
    ("contribution_republican_count_ratio","contribution_republican_count","contribution_count"),
    ("politician_challenger_ratio",  "politician_challenger_count",           "politician_count"),
    ("politician_democratic_ratio",  "politician_democratic_count",           "politician_count"),
    ("politician_incumbency_ratio",  "politician_incumbency_count",           "politician_count"),
    ("politician_open_pos_ratio",    "politician_open_pos_count",             "politician_count"),
    ("politician_republican_ratio",  "politician_republican_count",           "politician_count"),
    # dollar‑sum based
    ("governor_contribution_ratio",  "governor_contributions_sum_2010_usd",   "contribution_sum_2010_usd"),
    ("house_and_assembly_contribution_ratio","house_and_assembly_contributions_sum_2010_usd","contribution_sum_2010_usd"),
    ("senate_contribution_ratio",    "senate_contributions_sum_2010_usd",      "contribution_sum_2010_usd"),
    ("us_house_contribution_ratio",  "us_house_contributions_sum_2010_usd",    "contribution_sum_2010_usd"),
    ("us_senate_contribution_ratio", "us_senate_contributions_sum_2010_usd",   "contribution_sum_2010_usd"),
]

# 2) (Optional) Save the originals to compare later
orig_train = train[[old for old,_,_ in ratio_defs]].copy()
orig_test = test[[old for old,_,_ in ratio_defs]].copy()

# 3) Overwrite each ratio directly, safely
for oldcol, num, den in ratio_defs:
    train[oldcol] = (
        train[num]
            .div(train[den].replace({0: np.nan}))  # 0→NaN
            .fillna(0)                             # NaN→0
            .clip(0, 1)                            # clamp into [0,1]
    )
    test[oldcol] = (
        test[num]
            .div(test[den].replace({0: np.nan}))  # 0→NaN
            .fillna(0)                             # NaN→0
            .clip(0, 1)                            # clamp into [0,1]
    )


# 4) Print average absolute differences vs. originals
print("Average absolute difference between original and recomputed ratios for train and test data:")
for oldcol, _, _ in ratio_defs:
    if oldcol in orig_train:
        delta_train = (train[oldcol] - orig_train[oldcol]).abs().mean()
        delta_test = (test[oldcol] - orig_test[oldcol]).abs().mean()
        print(f"  {oldcol:40s} {delta_train:.5f}")
        print(f"  {oldcol:40s} {delta_test:.5f}")
        
    else:
        print(f"  {oldcol:40s} (original not present)")

# 5) Sanity‑check
assert np.isfinite(train[[old for old,_,_ in ratio_defs]]).all().all(), "Still found infinities!"
assert np.isfinite(test[[old for old,_,_ in ratio_defs]]).all().all(), "Still found infinities!"

assert ((train[[old for old,_,_ in ratio_defs]] >= 0) & (train[[old for old,_,_ in ratio_defs]] <= 1)).all().all(), "Ratios out of [0,1]!"
assert ((test[[old for old,_,_ in ratio_defs]] >= 0) & (test[[old for old,_,_ in ratio_defs]] <= 1)).all().all(), "Ratios out of [0,1]!"


print("✅ All ratios safely overwritten in place and checked.")


# 1. Check how many duplicate rows exist
num_duplicates_train = train.duplicated().sum()
num_duplicates_test = train.duplicated().sum()

print(f"🔍 Number of duplicate rows in train: {num_duplicates_train}")
print(f"🔍 Number of duplicate rows in test: {num_duplicates_test}")

# 2. Drop duplicate records
train = train.drop_duplicates()
test = test.drop_duplicates()

print(f"✅ Duplicates removed for train. New shape: {train.shape}")
print(f"✅ Duplicates removed for test. New shape: {test.shape}")

import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from scipy.stats.mstats import winsorize

# ------------------------------------------------------------
# 0.  Lists from above skew table
# ------------------------------------------------------------
money_like = [
    'contribution_republican_sum_2010_usd','governor_contributions_sum_2010_usd',
    'contribution_republican_sum_2010_usd','contribution_sum_2010_usd',
    'senate_contributions_sum_2010_usd','house_and_assembly_contributions_sum_2010_usd',
    'us_senate_contributions_sum_2010_usd','contribution_democratic_sum_2010_usd',
    'us_house_contributions_sum_2010_usd'
]

count_like = [
    'contribution_republican_count','contribution_count',
    'contribution_democratic_count','candidacy_democratic_count',
    'candidacy_republican_count','candidacy_count',
    'politician_open_pos_count','politician_republican_count',
    'politician_incumbency_count','politician_democratic_count',
    'politician_count','politician_challenger_count'
]

ratio_like = [
    'us_senate_contribution_ratio','us_house_contribution_ratio',
    'politician_challenger_ratio','senate_contribution_ratio','state_all_comm'
]

# ------------------------------------------------------------
# 1.  log1p transform for money & count cols
# ------------------------------------------------------------
for col in money_like + count_like:
    if col in train.columns:           # guard against typos / dropped cols
        train[col] = np.log1p(train[col].clip(lower=0))
        test[col]  = np.log1p(test[col].clip(lower=0))

# ------------------------------------------------------------
# 2.  Yeo‑Johnson power transform for skewed ratios
# ------------------------------------------------------------
pt = PowerTransformer(method="yeo-johnson", standardize=False)
train[ratio_like] = pt.fit_transform(train[ratio_like])
test[ratio_like]  = pt.transform(test[ratio_like])

# ------------------------------------------------------------
# 3.  Winsorise: clip any extreme 1 % tails on ALL numeric cols
# ------------------------------------------------------------
numeric_cols = train.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if col == "winner_ratio":
        lo, hi = np.percentile(train[col], [1, 99])
        train[col] = train[col].clip(lo, hi)
    else:
        lo, hi = np.percentile(train[col], [1, 99])
        train[col] = train[col].clip(lo, hi)
        test[col]  = test[col].clip(lo, hi)

print("✅  Skew & kurtosis treatments applied for train and test.")

import numpy as np
import pandas as pd
from category_encoders import TargetEncoder

# ------------------------------------------------------------------
# 1.  PARAMETERS
# ------------------------------------------------------------------
TOP_GENERAL   = 25
TOP_SPECIFIC  = 100
STATE_MIN     = 200          # states with <200 rows → OTHER
TE_SMOOTHING  = 10           # strength of shrinkage in TargetEncoder

cat_top_lump  = {
    "general_sector"  : (TOP_GENERAL,  "Other_Sector"),
    "specific_sector" : (TOP_SPECIFIC, "Other_Specific"),
}

# ------------------------------------------------------------------
# 2.  HELPER FUNCTIONS
# ------------------------------------------------------------------
def lump_by_top(df, col, top_n, new_label):
    top_vals = df[col].value_counts().nlargest(top_n).index
    df[col]  = df[col].where(df[col].isin(top_vals), new_label)
    return df

def lump_by_min(df, col, min_cnt, new_label):
    small = df[col].value_counts()[lambda s: s < min_cnt].index
    df[col] = df[col].where(~df[col].isin(small), new_label)
    return df

# ------------------------------------------------------------------
# 3.  APPLY LUMPING (***train and test***)
# ------------------------------------------------------------------
for col, (top_n, label) in cat_top_lump.items():
    train = lump_by_top(train, col, top_n, label)
    test  = lump_by_top(test,  col, top_n, label)   # ★ keep mapping identical

train = lump_by_min(train, "state", STATE_MIN, "OTHER")
test  = lump_by_min(test,  "state", STATE_MIN, "OTHER")

# ------------------------------------------------------------------
# 4.  TARGET ENCODING  (***fit on train, transform test***)
# ------------------------------------------------------------------
te_cols = ["general_sector", "specific_sector", "state"]
te      = TargetEncoder(cols=te_cols, smoothing=TE_SMOOTHING)

train[te_cols] = te.fit_transform(train[te_cols], train["winner_ratio"])
test[te_cols]  = te.transform(test[te_cols])            # ★ no fit here

# ------------------------------------------------------------------
# 5  FREQUENCY‑ENCODE city and zip_code  (train‑fit, test‑transform)
# ------------------------------------------------------------------
for col in ["city", "zip_code"]:
    freq = (train[col].astype(str)      # ensure consistent dtype
                    .value_counts(normalize=True))   # share of total rows

    train[f"{col}_freq"] = train[col].astype(str).map(freq).fillna(0)
    test[f"{col}_freq"]  = test[col].astype(str).map(freq).fillna(0)

# drop the raw string columns (high cardinality)
train.drop(columns=["city", "zip_code"], inplace=True)
test.drop(columns=["city",  "zip_code"], inplace=True)


# ------------------------------------------------------------------
# 6.  HANDLE THE REMAINING SMALL CATEGORICALS WITH ONE‑HOT
# ------------------------------------------------------------------
cat_small = ["contributor_type"]      # only 3 levels
train = pd.get_dummies(train, columns=cat_small, drop_first=True)
test  = pd.get_dummies(test,  columns=cat_small, drop_first=True)

# Keep column order identical in train & test after one‑hot:
missing_cols = [c for c in train.columns if c not in test.columns]
for c in missing_cols:
    print(f'{c} column is missing in test data')
    test[c] = 0
test = test[train.columns.drop("winner_ratio")]         # align

print("✅  Pre‑processing finished. Shapes:", train.shape)
print("✅  Pre‑processing finished. Shapes:",   test.shape)
   
for col in ["city", "zip_code","contributor_type"]:
    if col in cat_vars:
        cat_vars.remove(col)


/var/folders/h5/yfvk0l4921v0kg94n8qt42wc0000gn/T/ipykernel_65768/3694093064.py:1: DtypeWarning: Columns (31,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("./train_data_new.csv")


Average absolute difference between original and recomputed ratios for train and test data:
  candidacy_democratic_ratio               0.00000
  candidacy_democratic_ratio               0.00000
  candidacy_republican_ratio               0.00000
  candidacy_republican_ratio               0.00000
  contribution_democratic_count_ratio      0.00000
  contribution_democratic_count_ratio      0.00000
  contribution_republican_count_ratio      0.00000
  contribution_republican_count_ratio      0.00000
  politician_challenger_ratio              0.00000
  politician_challenger_ratio              0.00000
  politician_democratic_ratio              0.00000
  politician_democratic_ratio              0.00000
  politician_incumbency_ratio              0.00000
  politician_incumbency_ratio              0.00000
  politician_open_pos_ratio                0.00000
  politician_open_pos_ratio                0.00000
  politician_republican_ratio              0.00000
  politician_republican_ratio            

In [9]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

# ── 1) FEATURES & TARGET ──────────────────────────────────────────────────────
FEATURES = [
    'politician_incumbency_ratio',
    'politician_incumbency_count',
    'politician_challenger_ratio',
    'politician_open_pos_ratio',
    'politician_challenger_count',
    'politician_open_pos_count',
    'governor_contribution_ratio',
    'contribution_sum_2010_usd',
    'candidacy_count',
    'house_and_assembly_contributions_sum_2010_usd',
    'senate_contributions_sum_2010_usd',
    'senate_contribution_ratio',
    'governor_contributions_sum_2010_usd',
    'house_and_assembly_contribution_ratio',
    'politician_count',
    'candidacy_republican_count',
    'politician_republican_count',
    'contribution_republican_count',
    'contribution_count',
    'contribution_republican_sum_2010_usd'
]

X = train[FEATURES]
y = train["winner_ratio"]
X_test = test[FEATURES]

In [11]:
# ------------------------------------------
# 0.  PREP
# ------------------------------------------
import numpy as np, pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# X  : all engineered features (numeric dataframe)
# y  : target
X = train.drop(columns=["winner_ratio"])
y = train["winner_ratio"].values.astype("float32")

# ------------------------------------------
# 1.  STANDARDISE  (fit only on train folds)
# ------------------------------------------
scaler = StandardScaler()

# ------------------------------------------
# 2.  MODEL BUILDER
# ------------------------------------------
def make_model(input_dim, hidden=[256,128,64], drop=0.20):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    for h in hidden:
        model.add(layers.Dense(h, activation="relu"))
        model.add(layers.Dropout(drop))
    model.add(layers.Dense(1, activation="sigmoid"))       # ⬅️ ensures 0‑1
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="mse",                                        # Kaggle metric
        metrics=[],
    )
    return model

# ------------------------------------------
# 3.  K‑FOLD TRAINING
# ------------------------------------------
N_SPLITS = 5
BATCH    = 256
EPOCHS   = 300

kf   = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
oof  = np.zeros(len(X))
pred = np.zeros(len(test))          # will accumulate test predictions

for fold, (tr_idx, val_idx) in enumerate(kf.split(X), 1):
    print(f"\n📂 Fold {fold}")

    # 3‑a  fit scaler on *training* rows only
    X_tr = scaler.fit_transform(X.iloc[tr_idx])
    X_va = scaler.transform(X.iloc[val_idx])

    # 3‑b  build / train
    net = make_model(X_tr.shape[1])
    es  = keras.callbacks.EarlyStopping(
            patience=20, restore_best_weights=True, verbose=0)
    net.fit(X_tr, y[tr_idx],
            validation_data=(X_va, y[val_idx]),
            epochs=EPOCHS, batch_size=BATCH,
            callbacks=[es], verbose=0)

    # 3‑c  out‑of‑fold preds & score
    oof[val_idx] = net.predict(X_va, batch_size=BATCH).squeeze()
    y_tr_pred = net.predict(X_tr, batch_size=BATCH).squeeze()
    y_val_pred = net.predict(X_va, batch_size=BATCH).squeeze()

    train_mse = mean_squared_error(y[tr_idx], y_tr_pred)
    val_mse = mean_squared_error(y[val_idx], y_val_pred)

    oof[val_idx] = y_val_pred

    print(f"   fold {fold} TRAIN MSE = {train_mse:.6f}")
    print(f"   fold {fold} VAL   MSE = {val_mse:.6f}")

    mse = mean_squared_error(y[val_idx], oof[val_idx])
    print(f"   fold {fold} MSE = {mse:.6f}")

    # 3‑d  test predictions (scale test once per fold)
    X_test_scaled = scaler.transform(test[X.columns])
    pred += net.predict(X_test_scaled, batch_size=BATCH).squeeze() / N_SPLITS

print("\n📈 CV  MSE  =", mean_squared_error(y, oof))

# ------------------------------------------
# 4.  SUBMISSION
# ------------------------------------------
sub = pd.DataFrame({"index": test.index, "winner_ratio": pred.clip(0,1)})
sub.to_csv("nn_sigmoid_submission.csv", index=False)
print("Submission saved: nn_sigmoid_submission.csv")



📂 Fold 1
Metal device set to: Apple M4

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2025-04-17 11:28:11.645317: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-17 11:28:11.645438: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-04-17 11:28:11.966971: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2025-04-17 11:28:12.139650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2025-04-17 11:28:14.022608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 0s 1ms/step


2025-04-17 11:31:41.324716: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


134/134 [==============================] - 0s 979us/step
   fold 1 TRAIN MSE = 0.076006
   fold 1 VAL   MSE = 0.085509
   fold 1 MSE = 0.085509
451/451 [==============================] - 0s 1ms/step

📂 Fold 2


2025-04-17 11:31:43.427362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2025-04-17 11:31:45.267311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [ ]:
# # Try a gradient‑boosting model built for categorical + numerical**<br> CatBoost 
# # (or LightGBM w/ categorical) usually wins on these “messy mix” tables. 
# # | Handles remaining categories natively, robust to skew, requires little tuning. 

# from catboost import CatBoostRegressor, Pool

# cat_cols = ["contributor_type"]           # now the only remaining string col
# train_pool = Pool(data=train.drop("winner_ratio", axis=1),
#                   label=train["winner_ratio"],
#                   cat_features=[train.columns.get_loc(c) for c in cat_cols])
# model = CatBoostRegressor(
#         depth=6, learning_rate=0.05, iterations=3000,
#         loss_function="RMSE", eval_metric="RMSE",
#         early_stopping_rounds=200, verbose=200)
# model.fit(train_pool, eval_set=train_pool)   # quick CV → use KFold for final

: 

: 